# Определение теплоты испарения жидкости

In [38]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from scipy.optimize import minimize
from matplotlib.backends.backend_pgf import FigureCanvasPgf
matplotlib.backend_bases.register_backend('pdf', FigureCanvasPgf)

matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'text.usetex': True,
    'font.family': 'serif',
    'pgf.rcfonts': False,
})

def read_files(files):
    data = []
    for f in files:
        data += [pd.read_csv(f, sep=',', skipinitialspace=True)]
    return data

def get_linear(x, y, xerr, yerr):
    [a, b], cov = np.polyfit(x, y, deg=1, cov=True)
    aerr = np.sqrt(np.diag(cov))[0]
    syserr = np.sqrt((np.average(xerr) / (max(x) - min(x)))**2
                         + (np.average(yerr) / (max(y) - min(y)))**2)
    a_syserr = a * syserr
    berr = aerr * np.sqrt(np.average([it**2 for it in x]))
    b_syserr = b * syserr
    return ((a, b), (np.sqrt(aerr**2 + a_syserr**2), np.sqrt(berr**2 + b_syserr**2)))

def fit(f, params, x, y):
    def err(par, x_, y_):
        y1 = f(par, x_)
        return np.sum((y1 - y_)**2)

    return minimize(err, x0=params, args=(x, y)).x

# par[0] = L
def pressure(par, T):
    return par[1] * np.exp(par[0] / 8.31 * (par[2] - 1 / (T + 273)))

def dp(par, T):
    return par[0] / 8.31 * 1 / ((T + 273)**2) * pressure(par, T)

labels = ['Heating', 'Cooling']

[data, cooling] = read_files(['data/data.csv', 'data/cooling.csv'])
colors = ['red', 'green']

plt.figure(figsize=(7, 4))

i = 0
for it in [data, cooling]:
    inverse_T = it['1/T']
    ln_P = it['lnP']
    plt.errorbar(inverse_T, ln_P, xerr=0, yerr=0, fmt=f'{colors[i][0]}.')
    ((a, b), (aerr, berr)) = get_linear(inverse_T, ln_P, [0], [0])
    print(f'a = {a:.2f}, sigma = {aerr:.2f}')
    z = np.linspace(min(inverse_T), max(inverse_T), 1000)
    plt.plot(z, a * z + b, color=colors[i], label=labels[i])
    i += 1

# Оформление
plt.xlabel('$1 / T$, $10^{-3}$ K$^{-1}$')
plt.ylabel('$\\ln P$')
plt.grid(linestyle='--')
plt.legend()
plt.savefig('data/lin.pdf')

plt.figure(figsize=(7, 4))

i = 0
params = [0] * 3
for it in [data, cooling]:
    T = it['T']
    P = it['P']
    plt.errorbar(T, P, xerr=0.1, yerr=0.1, fmt=f'{colors[i][0]}.')
    params = fit(pressure, params, T, P)

    z = np.linspace(min(T), max(T), 1000)
    if i == 0:
        # Tangents
        start = min(T)
        end = max(T)
        plt.plot(z, dp(params, start) * (z - start) + pressure(params, start), color='red')
        plt.plot(z, dp(params, end) * (z - end) + pressure(params, end), color='red')
        plt.ylim((40, 140))
    print(f'L = {params[0]:.0f}')
    plt.plot(z, pressure(params, z), color=colors[i], label=labels[i])
    i += 1

# Оформление
plt.xlabel('$T$, \\textdegree C')
plt.ylabel('$P$, mm hg')
plt.grid(linestyle='--')
plt.legend()
plt.savefig('data/curve.pdf')


a = -5.32, sigma = 0.10
a = -4.27, sigma = 0.23
L = 43601
L = 43601
